## Klong beam simulation with trajectories
The Klong_beam notebook shows the results of a 5B event simulation of the KLF photon beam incident on the beryllium conversion target. That study is useful for counting the flux of particles that reach the GlueX liquid hydrogen target, but tells me nothing about where the particles that reach the target where produced, either for the klongs or the backgrounds. For that, I need to turn on the TRAJECTORIES card in the simulation control.in, and accept the copious amount of output that it produces. For just 100k beam photons running with "TRAJ 2", the size of the output hddm file is 35GB. Running a full sample of 5B events with "TRAJ 2" would occupy 1.7PB just to hold the output hddm data. This could be compressed by a factor 2-3, but even so it is too much for my storage resources, and trying to read through all of those data for analysis would take far too long. In this study, I ran just 200k events with "TRAJ 2", with my phi(1020)->KL,KS enhancement factor set to x100K so I have some chance of seeing klongs in the ouptut.

In [1]:
import ROOT
%jsroot on
#%pip install --user gluex.hddm_s==2.2.1
from gluex import hddm_s
from gluex import xrootd_client as xclient
import numpy as np

In [2]:
import dask.distributed
import dask
client = dask.distributed.Client(n_workers=50, threads_per_worker=1, dashboard_address='localhost:8787')

In [3]:
deg = np.pi / 180 # radians
thetacut = 3 / 2300 # radians
klong_ctau = 51e-9 * 3e10 # cm
mass_klong = 0.497 # GeV/c^2
mass_proton = 0.93827208943 # GeV/c^2
mass_neutron = 0.93956542194 # GeV/c^2
mass_deuteron = mass_proton + mass_neutron - 2.2e-3 # GeV/c^2

In [4]:
xrdurl = "root://cn445.storrs.hpc.uconn.edu"
xrdpath = "/Gluex/resilient/simulation/KLFbeam-8-2024/"
inforced = xrdurl + xrdpath + "KLFbeam2_500k_{0}.hddm"
infile = xrdurl + xrdpath + "KLFbeam2_{0}.hddm"

In [5]:
histos = {}
canvases = {}
unique_names = {}
mechanisms = {}

def get_unique_name(prefix):
    suffix = 0
    while f"{prefix}{suffix}" in unique_names:
        suffix += 1
    unique_name = f"{prefix}{suffix}"
    unique_names[unique_name] = 1
    return unique_name
    
def select_canvas(cname, width=600, height=500):
    if cname in canvases:
        canvases[cname].cd()
    else:
        canvases[cname] = ROOT.TCanvas(cname, "", width, height)
    return canvases[cname]

In [6]:
savedhistos = "Klong_trajectories.root"
fsaved = ROOT.TFile(savedhistos, "update")
def flush_saved_histos():
    global fsaved
    fsaved.Close()
    fsaved = ROOT.TFile(savedhistos, "update")

In [7]:
def klong_histos(infile, maxevents=1e99, skipevents=0):
    hphi = ROOT.TH1D("hphi", "Klong lab phi distribution", 90, -180, 180)
    hphi.GetXaxis().SetTitle("phi (deg)")
    hphi.GetYaxis().SetTitle("counts")
    htheta = ROOT.TH1D("htheta", "Klong lab theta distribution", 1000, 0, 10)
    htheta.GetXaxis().SetTitle("theta (deg)")
    htheta.GetYaxis().SetTitle("counts")
    hke = ROOT.TH1D("hke", "Klong lab energy", 120, 0, 12)
    hke.GetXaxis().SetTitle("total energy (GeV)")
    hke.GetYaxis().SetTitle("counts")
    horig = ROOT.TH1D("horig", "Klong production z", 2500, -2500, 0)
    horig.GetXaxis().SetTitle("vertex z (cm)")
    horig.GetYaxis().SetTitle("counts")
    istream = hddm_s.istream(infile)
    istream.skip(skipevents)
    nevents = 0
    for rec in istream:
        for vtx in rec.getVertices():
            for pro in vtx.getProducts():
                if pro.pdgtype == 310:
                    for mom in pro.getMomenta():
                        phi = np.arctan2(mom.py, mom.px)
                        theta = np.arctan2((mom.px**2 + mom.py**2), mom.pz)
                        hphi.Fill(phi/deg)
                        htheta.Fill(theta/deg)
                        if theta < thetacut:
                            hke.Fill(mom.E)
                        for ori in vtx.getOrigins():
                            horig.Fill(ori.vz)
        nevents += 1
        if nevents % 10000 == 0:
            print(f"analyzed {nevents} events, continuing...")
        if nevents >= maxevents:
            break
    hphi.Write()
    htheta.Write()
    hke.Write()
    horig.Write()
    return (hphi, htheta, hke, horig)

In [18]:
cname = get_unique_name('c')
select_canvas(cname, width=1000, height=800)
try:
    h = [fsaved.Get(hname) for hname in ("hphi", "htheta", "hke", "horig")]
    [h[i].GetMaximum() for i in range(4)]
except:
    h = klong_histos(inforced.format(1), maxevents=10000, skip=0)
canvases[cname].Divide(2,2)
[(canvases[cname].cd(i+1), h[i].Draw()) for i in range(4)]
canvases[cname].cd(0)
canvases[cname].Draw()

In [19]:
cname = get_unique_name('c')
select_canvas(cname)
hzcoefname = h[3].GetName() + "fit"
hzcoef = h[3].Clone(hzcoefname)
hzcoef.GetXaxis().SetRangeUser(-2400, -2200)
hzcoef.Fit('expo', '', '', -2328, -2293)
hzcoef.SetMinimum(0)
hzcoef.SetMaximum(h[3].GetMaximum())
hzcoef.Draw()
canvases[cname].Draw()
canvases[cname].Update()

****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      50.7975
NDf                       =           33
Edm                       =  3.40128e-10
NCalls                    =           62
Constant                  =     -261.187   +/-   4.79        
Slope                     =    -0.114682   +/-   0.00206472  


In [20]:
def klong_trajs(infile, maxevents=1e99, skipevents=0):
    hbirth = ROOT.TH1D("hbirth", "Klong birth position", 2400, -2400, 0)
    hbirth.GetXaxis().SetTitle("z (cm)")
    hbirth.GetYaxis().SetTitle("counts")
    hdeath = ROOT.TH1D("hdeath", "Klong death position", 2400, -2400, 0)
    hdeath.GetXaxis().SetTitle("z (cm)")
    hdeath.GetYaxis().SetTitle("counts")
    hsprout = ROOT.TH1D("hsprout", "Klong sprout position", 2400, -2400, 0)
    hsprout.GetXaxis().SetTitle("z (cm)")
    hsprout.GetYaxis().SetTitle("counts")
    hdecay = ROOT.TH1D("hdecay", "Klong decay position", 2400, -2400, 0)
    hdecay.GetXaxis().SetTitle("z (cm)")
    hdecay.GetYaxis().SetTitle("counts")
    birthing = False
    istream = hddm_s.istream(infile)
    istream.skip(skipevents)
    nevents = 0
    for rec in istream:
        for pt in rec.getMcTrajectoryPoints():
            birthing = not birthing
            if pt.part == 10:
                if not pt.mech in mechanisms:
                    name = ''.join([chr((pt.mech >> i*8) % 256) for i in range(4)])
                    print(f"found new mech '{name}'")
                    mechanisms[pt.mech] = name
                if pt.mech == 0:
                    hsprout.Fill(pt.z)
                elif mechanisms[pt.mech] == "dcay":
                    hdecay.Fill(pt.z)
                if birthing:
                    hbirth.Fill(pt.z)
                else:
                    hdeath.Fill(pt.z)
        nevents += 1
        if nevents % 10000 == 0:
            print(f"analyzed {nevents} events, continuing...")
        if nevents >= maxevents:
            break
    return (hbirth, hdeath, hsprout, hdecay)

In [21]:
cname = get_unique_name('c')
select_canvas(cname, width=1000, height=800)
try:
    h = [fsaved.Get(hname) for hname in ("hbirth", "hdeath", "hsprout", "hdecay")]
    [h[i].GetMaximum() for i in range(4)]
except:
    results = [dask.delayed(klong_trajs)(inforced2.format(i)) for i in range(500)]
    def collector(results):
        for result in results:
            try:
                [hlist[i].Add(result[i]) for i in range(4)]
            except:
                hlist = result
        return hlist
    round2 = [dask.delayed(collector)(results[i*10:(i+1)*10]) for i in range(len(results)//10)]
    lastround = dask.delayed(collector)(round2)
    h = lastround.compute()
    [hist.Write() for hist in h]
canvases[cname].Divide(2,2)
[(canvases[cname].cd(i+1), h[i].Draw()) for i in range(4)]
canvases[cname].cd(0)
canvases[cname].Draw()

In [22]:
cname = get_unique_name('c')
select_canvas(cname, width=1200, height=600)
h = [fsaved.Get(hname) for hname in ("hbirth", "hdeath", "hsprout", "hdecay")]
h[0].SetMaximum(max([h[i].GetMaximum() for i in range(2)]) * 1.1)
[h[i].SetStats(0) for i in range(4)]
h[0].SetMinimum(1)
h[0].SetTitle('')
h[0].Draw()
h[1].SetLineColor(2)
h[1].Draw("same")
h[3].SetLineColor(3)
h[3].Draw("same")
canvases[cname].Draw()
canvases[cname].Update()

In [23]:
alcove_entry_wall_z = -2454.5
beconv_target_start_z = 125.6 + alcove_entry_wall_z
tungsten_plug_start_z = 166.1 + alcove_entry_wall_z
vacuum_window_z = 259.3 + alcove_entry_wall_z
lead_wall_start_z = 386.4 + alcove_entry_wall_z
second_wall_start_z = 482.8 + alcove_entry_wall_z
sweep_magnet_start_z = 602.6 + alcove_entry_wall_z
exit_wall_start_z = 1004.9 + alcove_entry_wall_z
ps_magnet_start_z = 1270.7 + alcove_entry_wall_z
print(f"Beryllium converter: z = {beconv_target_start_z:.0f} cm")
print(f"Tungsten plug: z = {tungsten_plug_start_z:.0f} cm")
print(f"Vacuum begins: z = {vacuum_window_z:.0f} cm")
print(f"Lead wall: z = {lead_wall_start_z:.0f} cm")
print(f"Second concrete wall: z = {second_wall_start_z:.0f} cm")
print(f"Sweep magnet start: z = {sweep_magnet_start_z:.0f} cm")
print(f"Alcove exit wall: z = {exit_wall_start_z:.0f} cm")
print(f"Pair spectrometer: z = {ps_magnet_start_z:.0f} cm")

Beryllium converter: z = -2329 cm
Tungsten plug: z = -2288 cm
Vacuum begins: z = -2195 cm
Lead wall: z = -2068 cm
Second concrete wall: z = -1972 cm
Sweep magnet start: z = -1852 cm
Alcove exit wall: z = -1450 cm
Pair spectrometer: z = -1184 cm


## Look at the 5B sample

In [24]:
def klong_vertices(infile, maxevents=1e99, skipevents=0):
    hvertz = ROOT.TH1D("hvertz", "Klong production vertex z", 24000, -2400, 0)
    hvertz.GetXaxis().SetTitle("z (cm)")
    hvertz.GetYaxis().SetTitle("counts")
    hvertE = ROOT.TH1D("hvertE", "Klong production total energy", 250, 0, 10)
    hvertE.GetXaxis().SetTitle("E (GeV)")
    hvertE.GetYaxis().SetTitle("counts")
    try:
        istream = hddm_s.istream(infile)
        istream.skip(skipevents)
    except:
        return (hvertz, hvertE)
    nevents = 0
    for rec in istream:
        for vtx in rec.getVertices():
            for pro in vtx.getProducts():
                if pro.pdgtype == 130:
                    for ori in vtx.getOrigins():
                        hvertz.Fill(ori.vz)
                    for mom in pro.getMomenta():
                        hvertE.Fill(mom.E)
        nevents += 1
        if nevents >= maxevents:
            break
    return (hvertz, hvertE)

In [25]:
cname = get_unique_name('c')
select_canvas(cname, width=1200, height=500)
try:
    h = [fsaved.Get(hname) for hname in ("hvertz", "hvertE")]
    [h[i].GetMaximum() for i in range(2)]
except:
    print("histograms hvertz, hvertE not found, regenerating...")
    results = [dask.delayed(klong_vertices)(infile.format(i+1)) for i in range(10000)]
    def collector(results):
        for result in results:
            try:
                [hlist[i].Add(result[i]) for i in range(2)]
            except:
                hlist = result
        return hlist
    round2 = [dask.delayed(collector)(results[i*100:(i+1)*100]) for i in range(len(results)//100)]
    lastround = dask.delayed(collector)(round2)
    h = lastround.compute()
    [hist.Write() for hist in h]
canvases[cname].Divide(2,1)
[(canvases[cname].cd(i+1), h[i].Draw()) for i in range(2)]
canvases[cname].cd(0)
canvases[cname].Draw()

In [26]:
cname = get_unique_name('c')
select_canvas(cname, width=1200, height=500)
try:
    h = [fsaved.Get(hname) for hname in ("hvertz2", "hvertE2")]
    [h[i].GetMaximum() for i in range(2)]
except:
    print("histograms hvertz2, hvertE2 not found, regenerating...")
    results = [dask.delayed(klong_vertices)(infile.format(i+1)) for i in range(10000)]
    def collector(results):
        for result in results:
            try:
                [hlist[i].Add(result[i]) for i in range(2)]
            except:
                hlist = result
        return hlist
    round2 = [dask.delayed(collector)(results[i*100:(i+1)*100]) for i in range(len(results)//100)]
    lastround = dask.delayed(collector)(round2)
    h = lastround.compute()
    h[0].SetName("hvertz2")
    h[1].SetName("hvertE2")
    [hist.Write() for hist in h]
canvases[cname].Divide(2,1)
[(canvases[cname].cd(i+1), h[i].Draw()) for i in range(2)]
canvases[cname].cd(0)
canvases[cname].Draw()

In [27]:
flush_saved_histos()